In [1]:
import os
import re
import pandas as pd
import spacy
import en_core_web_sm
# from spacy import doc

In [2]:
def extract_chunks(input_text):
    doc = nlp(input_text)
    spans = list(doc.ents)  #+ list(doc.noun_chunks)
    out_list = []
    for span in spans:
        span.merge()
        out_list.append(span)
    return out_list


In [3]:
def extract_name_pair(name, input_text):
    out_list = []
    if not pd.isnull(input_text):
#         s_out = extract_chunks(input_text)    
        doc = nlp(input_text)
        spans = list(doc.ents)  #+ list(doc.noun_chunks)

        for item in spans:
            try:
                item = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=item.string)            
                if (re.search(item,name,flags=re.IGNORECASE)):
                    out_list.append(item)

            except:
                pass
            
    return out_list
#     else:
#         return None

In [4]:
def basic_wc(input_text):
    if not (pd.isnull(input_text)):
        sent = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=input_text)            
        doc = nlp(sent)
        return doc.__len__()
    else:
        return 0


In [7]:
nlp = en_core_web_sm.load()
for ii in range(2,91980):
    try:
        # out_analyst= pd.read_csv('analyst_out_foringestion.csv',encoding='utf-8',na_values='NaN')
        filename = 'extract/analyst_out_foringestion_'+str(ii)+'.csv' 
        fileout = 'transform/analyst_out_withcount_'+str(ii)+'.csv' 

        out_analyst= pd.read_csv(filename,encoding='utf-8',na_values='NaN',header=0,\
                                 names=['index','file','quarter_year','executive_firm','ticker','analyst_name',\
                                       'analyst_firm','analyst_content','analyst_qanda','exec_name','exec_title'])

        out_analyst['exec_mention_qna'] = out_analyst.apply(lambda x: extract_name_pair(x['exec_name'],x['analyst_qanda']), axis=1)
        out_analyst['exec_mention_transcript'] = out_analyst.apply(lambda x: extract_name_pair(x['exec_name'],x['analyst_content']), axis=1)
        out_analyst['exec_count_qna'] = out_analyst.apply(lambda x: \
                                                 len(x['exec_mention_qna']), axis=1)
        out_analyst['exec_count_transcript'] = out_analyst.apply(lambda x: \
                                                 len(x['exec_mention_transcript']), axis=1)
        out_analyst['exec_count_total']= out_analyst.exec_count_qna+out_analyst.exec_count_transcript
        out_analyst['analyst_qna_count'] = out_analyst.apply(lambda x: \
                                                 basic_wc(x['analyst_qanda']), axis=1)
        out_analyst['analyst_content_count'] = out_analyst.apply(lambda x: \
                                                 basic_wc(x['analyst_content']), axis=1)
        out_analyst['analyst_total_wc'] = out_analyst['analyst_qna_count'] + out_analyst['analyst_content_count']

        # out_analyst[['file','quarter_year','executive_firm','ticker',\
        # 'analyst_name','analyst_firm','analyst_total_wc','exec_name','exec_title','exec_mention_qna','exec_count_qna',
        #             'exec_mention_transcript','exec_count_transcript','exec_count_total']]\
        # .to_csv('analyst_out_withcount.csv',sep=',', encoding = 'utf-8')
        # out_executives.select({[}'file','quarter_yer'])

        out_analyst[['file','quarter_year','executive_firm','ticker',\
        'analyst_name','analyst_firm','analyst_total_wc','exec_name','exec_title','exec_count_total']]\
        .to_csv(fileout,sep=',', encoding = 'utf-8')
    #     .to_csv('analyst_out_withcount.csv',sep=',', encoding = 'utf-8')    
    except:
        print(ii,'##fail')
        pass

6 ##fail
7 ##fail
20 ##fail
22 ##fail
23 ##fail
25 ##fail
29 ##fail
39 ##fail
41 ##fail
46 ##fail
48 ##fail
50 ##fail
56 ##fail
59 ##fail
77 ##fail
81 ##fail
95 ##fail
101 ##fail
113 ##fail
120 ##fail
141 ##fail
147 ##fail
156 ##fail
162 ##fail
171 ##fail
175 ##fail
200 ##fail
205 ##fail
226 ##fail
227 ##fail
236 ##fail
241 ##fail
257 ##fail
273 ##fail
283 ##fail
301 ##fail
302 ##fail
306 ##fail
313 ##fail
320 ##fail
322 ##fail
325 ##fail
337 ##fail
338 ##fail
339 ##fail
344 ##fail
349 ##fail
351 ##fail
360 ##fail
363 ##fail
372 ##fail
381 ##fail
402 ##fail
422 ##fail
426 ##fail
441 ##fail
465 ##fail
468 ##fail
473 ##fail
474 ##fail
483 ##fail
489 ##fail
496 ##fail
528 ##fail
532 ##fail
545 ##fail
547 ##fail
549 ##fail
569 ##fail
570 ##fail
581 ##fail
588 ##fail
610 ##fail
614 ##fail
616 ##fail
617 ##fail
633 ##fail
636 ##fail
639 ##fail
642 ##fail
650 ##fail
655 ##fail
663 ##fail
664 ##fail
668 ##fail
669 ##fail
684 ##fail
687 ##fail
703 ##fail
711 ##fail
712 ##fail
732 ##fail
753 ##f

5805 ##fail
5809 ##fail
5811 ##fail
5814 ##fail
5823 ##fail
5824 ##fail
5835 ##fail
5836 ##fail
5840 ##fail
5844 ##fail
5851 ##fail
5856 ##fail
5861 ##fail
5867 ##fail
5869 ##fail
5870 ##fail
5888 ##fail
5894 ##fail
5902 ##fail
5908 ##fail
5911 ##fail
5922 ##fail
5926 ##fail
5934 ##fail
5938 ##fail
5959 ##fail
5960 ##fail
5964 ##fail
5967 ##fail
5969 ##fail
5975 ##fail
5991 ##fail
5993 ##fail
5994 ##fail
6016 ##fail
6017 ##fail
6018 ##fail
6021 ##fail
6026 ##fail
6034 ##fail
6035 ##fail
6036 ##fail
6038 ##fail
6041 ##fail
6042 ##fail
6045 ##fail
6046 ##fail
6049 ##fail
6051 ##fail
6053 ##fail
6055 ##fail
6064 ##fail
6071 ##fail
6080 ##fail
6082 ##fail
6087 ##fail
6102 ##fail
6105 ##fail
6107 ##fail
6118 ##fail
6120 ##fail
6125 ##fail
6128 ##fail
6131 ##fail
6137 ##fail
6138 ##fail
6141 ##fail
6145 ##fail
6165 ##fail
6169 ##fail
6171 ##fail
6172 ##fail
6174 ##fail
6185 ##fail
6188 ##fail
6199 ##fail
6209 ##fail
6222 ##fail
6240 ##fail
6254 ##fail
6267 ##fail
6279 ##fail
6327 ##fail
6329

10906 ##fail
10907 ##fail
10913 ##fail
10915 ##fail
10918 ##fail
10930 ##fail
10938 ##fail
10948 ##fail
10962 ##fail
10982 ##fail
10986 ##fail
10995 ##fail
10997 ##fail
11004 ##fail
11019 ##fail
11023 ##fail
11030 ##fail
11033 ##fail
11050 ##fail
11060 ##fail
11061 ##fail
11064 ##fail
11071 ##fail
11074 ##fail
11084 ##fail
11109 ##fail
11120 ##fail
11139 ##fail
11152 ##fail
11155 ##fail
11165 ##fail
11202 ##fail
11219 ##fail
11221 ##fail
11223 ##fail
11228 ##fail
11235 ##fail
11254 ##fail
11261 ##fail
11279 ##fail
11281 ##fail
11289 ##fail
11296 ##fail
11304 ##fail
11307 ##fail
11312 ##fail
11318 ##fail
11321 ##fail
11333 ##fail
11341 ##fail
11355 ##fail
11373 ##fail
11378 ##fail
11393 ##fail
11398 ##fail
11404 ##fail
11410 ##fail
11411 ##fail
11413 ##fail
11426 ##fail
11428 ##fail
11446 ##fail
11463 ##fail
11464 ##fail
11471 ##fail
11483 ##fail
11495 ##fail
11509 ##fail
11512 ##fail
11517 ##fail
11529 ##fail
11541 ##fail
11554 ##fail
11562 ##fail
11584 ##fail
11597 ##fail
11616 ##fail

15355 ##fail
15364 ##fail
15367 ##fail
15372 ##fail
15377 ##fail
15391 ##fail
15394 ##fail
15410 ##fail
15411 ##fail
15415 ##fail
15419 ##fail
15470 ##fail
15474 ##fail
15478 ##fail
15484 ##fail
15508 ##fail
15518 ##fail
15523 ##fail
15535 ##fail
15536 ##fail
15541 ##fail
15542 ##fail
15543 ##fail
15546 ##fail
15589 ##fail
15591 ##fail
15596 ##fail
15602 ##fail
15614 ##fail
15616 ##fail
15618 ##fail
15631 ##fail
15637 ##fail
15643 ##fail
15647 ##fail
15652 ##fail
15679 ##fail
15688 ##fail
15697 ##fail
15698 ##fail
15701 ##fail
15706 ##fail
15711 ##fail
15725 ##fail
15729 ##fail
15743 ##fail
15745 ##fail
15767 ##fail
15770 ##fail
15783 ##fail
15789 ##fail
15804 ##fail
15821 ##fail
15823 ##fail
15826 ##fail
15828 ##fail
15832 ##fail
15856 ##fail
15858 ##fail
15861 ##fail
15865 ##fail
15877 ##fail
15884 ##fail
15887 ##fail
15896 ##fail
15899 ##fail
15904 ##fail
15906 ##fail
15907 ##fail
15910 ##fail
15911 ##fail
15913 ##fail
15918 ##fail
15924 ##fail
15930 ##fail
15933 ##fail
15948 ##fail

19960 ##fail
19964 ##fail
19976 ##fail
20003 ##fail
20016 ##fail
20039 ##fail
20049 ##fail
20050 ##fail
20052 ##fail
20054 ##fail
20058 ##fail
20062 ##fail
20070 ##fail
20077 ##fail
20083 ##fail
20086 ##fail
20111 ##fail
20114 ##fail
20120 ##fail
20148 ##fail
20149 ##fail
20155 ##fail
20168 ##fail
20181 ##fail
20187 ##fail
20192 ##fail
20198 ##fail
20202 ##fail
20211 ##fail
20218 ##fail
20227 ##fail
20228 ##fail
20237 ##fail
20239 ##fail
20260 ##fail
20270 ##fail
20273 ##fail
20280 ##fail
20287 ##fail
20288 ##fail
20291 ##fail
20300 ##fail
20302 ##fail
20329 ##fail
20333 ##fail
20345 ##fail
20346 ##fail
20348 ##fail
20350 ##fail
20371 ##fail
20390 ##fail
20393 ##fail
20403 ##fail
20405 ##fail
20413 ##fail
20422 ##fail
20428 ##fail
20432 ##fail
20446 ##fail
20449 ##fail
20452 ##fail
20459 ##fail
20460 ##fail
20462 ##fail
20471 ##fail
20483 ##fail
20484 ##fail
20508 ##fail
20509 ##fail
20512 ##fail
20518 ##fail
20523 ##fail
20525 ##fail
20528 ##fail
20557 ##fail
20559 ##fail
20577 ##fail

...

74907 ##fail
74913 ##fail
74914 ##fail
74915 ##fail
74918 ##fail
74919 ##fail
74922 ##fail
74923 ##fail
74934 ##fail
74940 ##fail
74941 ##fail
74942 ##fail
74944 ##fail
74960 ##fail
74961 ##fail
74965 ##fail
74966 ##fail
74982 ##fail
74989 ##fail
74992 ##fail
75005 ##fail
75006 ##fail
75011 ##fail
75021 ##fail
75022 ##fail
75032 ##fail
75038 ##fail
75039 ##fail
75040 ##fail
75045 ##fail
75052 ##fail
75056 ##fail
75066 ##fail
75072 ##fail
75075 ##fail
75078 ##fail
75083 ##fail
75086 ##fail
75088 ##fail
75096 ##fail
75103 ##fail
75107 ##fail
75108 ##fail
75112 ##fail
75113 ##fail
75114 ##fail
75116 ##fail
75130 ##fail
75132 ##fail
75136 ##fail
75149 ##fail
75152 ##fail
75153 ##fail
75157 ##fail
75176 ##fail
75185 ##fail
75197 ##fail
75202 ##fail
75205 ##fail
75212 ##fail
75226 ##fail
75232 ##fail
75240 ##fail
75241 ##fail
75250 ##fail
75253 ##fail
75254 ##fail
75257 ##fail
75270 ##fail
75279 ##fail
75283 ##fail
75284 ##fail
75293 ##fail
75294 ##fail
75297 ##fail
75302 ##fail
75304 ##fail

78412 ##fail
78422 ##fail
78427 ##fail
78428 ##fail
78429 ##fail
78430 ##fail
78438 ##fail
78449 ##fail
78455 ##fail
78457 ##fail
78458 ##fail
78461 ##fail
78463 ##fail
78466 ##fail
78467 ##fail
78472 ##fail
78473 ##fail
78475 ##fail
78485 ##fail
78489 ##fail
78494 ##fail
78498 ##fail
78500 ##fail
78501 ##fail
78504 ##fail
78508 ##fail
78523 ##fail
78552 ##fail
78561 ##fail
78578 ##fail
78581 ##fail
78585 ##fail
78590 ##fail
78596 ##fail
78597 ##fail
78601 ##fail
78608 ##fail
78611 ##fail
78612 ##fail
78619 ##fail
78620 ##fail
78631 ##fail
78632 ##fail
78635 ##fail
78638 ##fail
78644 ##fail
78648 ##fail
78653 ##fail
78654 ##fail
78656 ##fail
78663 ##fail
78669 ##fail
78671 ##fail
78675 ##fail
78681 ##fail
78684 ##fail
78692 ##fail
78706 ##fail
78709 ##fail
78716 ##fail
78731 ##fail
78734 ##fail
78735 ##fail
78736 ##fail
78743 ##fail
78749 ##fail
78756 ##fail
78761 ##fail
78764 ##fail
78765 ##fail
78769 ##fail
78776 ##fail
78794 ##fail
78805 ##fail
78810 ##fail
78817 ##fail
78824 ##fail

81865 ##fail
81866 ##fail
81872 ##fail
81878 ##fail
81886 ##fail
81887 ##fail
81892 ##fail
81899 ##fail
81903 ##fail
81906 ##fail
81907 ##fail
81912 ##fail
81913 ##fail
81916 ##fail
81917 ##fail
81927 ##fail
81935 ##fail
81950 ##fail
81952 ##fail
81968 ##fail
81974 ##fail
81975 ##fail
81982 ##fail
81986 ##fail
81998 ##fail
81999 ##fail
82006 ##fail
82020 ##fail
82026 ##fail
82028 ##fail
82029 ##fail
82031 ##fail
82037 ##fail
82047 ##fail
82048 ##fail
82050 ##fail
82051 ##fail
82060 ##fail
82063 ##fail
82065 ##fail
82066 ##fail
82068 ##fail
82071 ##fail
82087 ##fail
82088 ##fail
82089 ##fail
82092 ##fail
82093 ##fail
82094 ##fail
82096 ##fail
82100 ##fail
82101 ##fail
82102 ##fail
82107 ##fail
82111 ##fail
82114 ##fail
82120 ##fail
82122 ##fail
82131 ##fail
82136 ##fail
82150 ##fail
82162 ##fail
82163 ##fail
82164 ##fail
82166 ##fail
82167 ##fail
82174 ##fail
82183 ##fail
82188 ##fail
82204 ##fail
82209 ##fail
82210 ##fail
82226 ##fail
82234 ##fail
82240 ##fail
82244 ##fail
82248 ##fail

85606 ##fail
85607 ##fail
85612 ##fail
85616 ##fail
85626 ##fail
85627 ##fail
85628 ##fail
85635 ##fail
85639 ##fail
85640 ##fail
85648 ##fail
85651 ##fail
85653 ##fail
85680 ##fail
85682 ##fail
85694 ##fail
85700 ##fail
85706 ##fail
85709 ##fail
85718 ##fail
85719 ##fail
85740 ##fail
85751 ##fail
85756 ##fail
85757 ##fail
85760 ##fail
85765 ##fail
85766 ##fail
85776 ##fail
85779 ##fail
85780 ##fail
85789 ##fail
85791 ##fail
85792 ##fail
85793 ##fail
85796 ##fail
85797 ##fail
85799 ##fail
85800 ##fail
85802 ##fail
85803 ##fail
85804 ##fail
85805 ##fail
85826 ##fail
85828 ##fail
85836 ##fail
85842 ##fail
85853 ##fail
85883 ##fail
85888 ##fail
85890 ##fail
85896 ##fail
85907 ##fail
85910 ##fail
85916 ##fail
85929 ##fail
85930 ##fail
85941 ##fail
85947 ##fail
85957 ##fail
85960 ##fail
85965 ##fail
85975 ##fail
85976 ##fail
86005 ##fail
86014 ##fail
86017 ##fail
86019 ##fail
86032 ##fail
86035 ##fail
86048 ##fail
86052 ##fail
86054 ##fail
86055 ##fail
86060 ##fail
86068 ##fail
86077 ##fail

89319 ##fail
89328 ##fail
89338 ##fail
89344 ##fail
89347 ##fail
89351 ##fail
89361 ##fail
89374 ##fail
89381 ##fail
89390 ##fail
89399 ##fail
89425 ##fail
89431 ##fail
89434 ##fail
89435 ##fail
89441 ##fail
89457 ##fail
89485 ##fail
89490 ##fail
89506 ##fail
89514 ##fail
89516 ##fail
89518 ##fail
89523 ##fail
89526 ##fail
89527 ##fail
89536 ##fail
89549 ##fail
89586 ##fail
89593 ##fail
89594 ##fail
89597 ##fail
89602 ##fail
89613 ##fail
89636 ##fail
89638 ##fail
89643 ##fail
89647 ##fail
89657 ##fail
89674 ##fail
89680 ##fail
89689 ##fail
89691 ##fail
89698 ##fail
89701 ##fail
89702 ##fail
89716 ##fail
89729 ##fail
89733 ##fail
89759 ##fail
89763 ##fail
89770 ##fail
89796 ##fail
89797 ##fail
89798 ##fail
89815 ##fail
89831 ##fail
89845 ##fail
89847 ##fail
89867 ##fail
89885 ##fail
89896 ##fail
89897 ##fail
89905 ##fail
89908 ##fail
89914 ##fail
89934 ##fail
89939 ##fail
89944 ##fail
89946 ##fail
89949 ##fail
89953 ##fail
89958 ##fail
89971 ##fail
89981 ##fail
89985 ##fail
89996 ##fail

In [ ]:
import csv
fieldnames = ['index','fname', 'status']

with open('log_ner.csv', 'wb') as csvfile:
    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
#     logwriter.writerow(fieldnames)

 
    for ii in range(2,91980):
        try:
#             total_rows = len(file_list)
            row_start = ii*1
            row_end = (ii*1)+1
        #     print('from: ', row_start, ' to: ', row_end)
            filename = 'extract/analyst_out_foringestion_'+str(row_start)+'.csv' 
            fileout = 'transform/analyst_out_withcount_'+str(row_start)+'.csv' 
            
            nlp = en_core_web_sm.load()
            out_analyst= pd.read_csv(filename,encoding='utf-8',na_values='NaN',header=0,\
                                     names=['index','file','quarter_year','executive_firm','ticker','analyst_name',\
                                           'analyst_firm','analyst_content','analyst_qanda','exec_name','exec_title'])
            out_analyst['exec_mention_qna'] = out_analyst.apply(lambda x: extract_name_pair(x['exec_name'],x['analyst_qanda']), axis=1)
            out_analyst['exec_mention_transcript'] = out_analyst.apply(lambda x: extract_name_pair(x['exec_name'],x['analyst_content']), axis=1)
            out_analyst['exec_count_qna'] = out_analyst.apply(lambda x: \
                                                     len(x['exec_mention_qna']), axis=1)
            out_analyst['exec_count_transcript'] = out_analyst.apply(lambda x: \
                                                     len(x['exec_mention_transcript']), axis=1)
            out_analyst['exec_count_total']= out_analyst.exec_count_qna+out_analyst.exec_count_transcript
            out_analyst['analyst_qna_count'] = out_analyst.apply(lambda x: \
                                                     basic_wc(x['analyst_qanda']), axis=1)
            out_analyst['analyst_content_count'] = out_analyst.apply(lambda x: \
                                                     basic_wc(x['analyst_content']), axis=1)
            out_analyst['analyst_total_wc'] = out_analyst['analyst_qna_count'] + out_analyst['analyst_content_count']

            # out_analyst[['file','quarter_year','executive_firm','ticker',\
            # 'analyst_name','analyst_firm','analyst_total_wc','exec_name','exec_title','exec_mention_qna','exec_count_qna',
            #             'exec_mention_transcript','exec_count_transcript','exec_count_total']]\
            # .to_csv('analyst_out_withcount.csv',sep=',', encoding = 'utf-8')
            # out_executives.select({[}'file','quarter_yer'])
            logwriter.writerow([ii,out_analyst.file.head(1).item(),'success'])

            out_analyst[['file','quarter_year','executive_firm','ticker',\
            'analyst_name','analyst_firm','analyst_total_wc','exec_name','exec_title','exec_count_total']]\
            .to_csv(fileout,sep=',', encoding = 'utf-8')           
#             logwriter.writerow([ii,'success'])

        except:
            print('exception: ', ii, ' ###')
#             logwriter.writerow([ii,'fail'])
            pass

exception:  2  ###
exception:  3  ###
exception:  4  ###
exception:  5  ###
exception:  6  ###
exception:  7  ###
exception:  8  ###
exception:  9  ###
exception:  10  ###
exception:  11  ###
exception:  12  ###
exception:  13  ###
exception:  14  ###
exception:  15  ###
exception:  16  ###
exception:  17  ###
exception:  18  ###
exception:  19  ###
exception:  20  ###
exception:  21  ###
exception:  22  ###
exception:  23  ###
exception:  24  ###
exception:  25  ###
exception:  26  ###
exception:  27  ###
exception:  28  ###
exception:  29  ###
exception:  30  ###
exception:  31  ###
exception:  32  ###
exception:  33  ###
exception:  34  ###
exception:  35  ###
exception:  36  ###
exception:  37  ###
exception:  38  ###
exception:  39  ###
exception:  40  ###
exception:  41  ###
exception:  42  ###
exception:  43  ###
exception:  44  ###
exception:  45  ###
exception:  46  ###
exception:  47  ###
exception:  48  ###
exception:  49  ###
exception:  50  ###
exception:  51  ###
exceptio